In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json

In [ ]:
resnet_fpn = json.load(open("resnet50-FPN-metrics.txt"))
densenet_fpn = json.load(open("densenet201-FPN-metrics.txt"))
mobilenet_fpn = json.load(open("mobilenet_v2-FPN-metrics.txt"))
resnext_fpn = json.load(open('se_resnext101_32x4d-FPN-metrics.txt'))

resnet_unet = json.load(open("resnet50-Unet-metrics.txt"))
densenet_unet = json.load(open("densenet201-Unet-metrics.txt"))
mobilenet_unet = json.load(open("mobilenet_v2-Unet-metrics.txt"))
resnext_unet = json.load(open('se_resnext101_32x4d-Unet-metrics.txt'))

mobilenet_unetpp = json.load(open('mobilenet_v2-UnetPlusPlus-metrics.txt'))
resnet_unetpp = json.load(open('resnet50-UnetPlusPlus-metrics.txt'))
densenet_unetpp = json.load(open('densenet201-UnetPlusPlus-metrics.txt'))
resnext_unetpp = json.load(open('se_resnext101_32x4d-UnetPlusPlus-metrics.txt'))

mobilenet_manet = json.load(open('mobilenet_v2-MAnet-metrics.txt'))
resnet_manet = json.load(open('resnet50-MAnet-metrics.txt'))
densenet_manet = json.load(open('densenet201-MAnet-metrics.txt'))
resnext_manet = json.load(open('se_resnext101_32x4d-MAnet-metrics.txt'))

In [ ]:
arch_dict = {'MobilenetV2 FPN':mobilenet_fpn, 'Resnet50 FPN':resnet_fpn, 'Densenet201 FPN':densenet_fpn, 
             'Resnext101 FPN': resnext_fpn, 'MobilenetV2 Unet':mobilenet_unet, 'Resnet50 Unet':resnet_unet, 
             'Densenet201 Unet':densenet_unet, 'Resnext101 Unet': resnext_unet, 'MobilenetV2 Unet++': mobilenet_unetpp,
             'Resnet50 Unet++': resnet_unetpp, 'Densenet201 Unet++': densenet_unetpp, 'Resnext101 Unet++': resnext_unetpp,
             'MobilenetV2 MAnet': mobilenet_manet, 'Resnet50 MAnet': resnet_manet, 'Densenet201 MAnet': densenet_manet,
             'Resnext101 MAnet': resnext_manet}

## Boxplot

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

#fig = plt.figure(figsize =(15, 13))

metrics = ['Accuracy', 'F1-Score', 'Dataset IoU', 'Per Image IoU', 'Hausdorff']

for metric in metrics:
    
    metric_list = []
    for arch in arch_dict:
        metric_list.append(arch_dict[arch][metric])
    
    fig, ax = plt.subplots()
    ax.boxplot(metric_list)
    ax.set_xticklabels(list(arch_dict.keys()), rotation=45, ha='right')
    #plt.ylim(0.6, 1)
    plt.title(metric)
    plt.show()

## Shapiro

In [ ]:
from scipy.stats import shapiro

print('-------------------------------------------------------------')
for metric in metrics:
    for arch in arch_dict:
        #print(arch_dict[arch][metric])
        result = shapiro(arch_dict[arch][metric])

        print(f'shapiro test for {arch} and {metric}')
        print(result)

        if result[1] < 0.05:
            print("rejected null hypothesis")
        else:
            print("did not rejected null hypothesis")

        print('------------------------------------------------------------- \n')

## Mann-Whitney

In [ ]:
from scipy.stats import mannwhitneyu
import scipy.stats as stats
import itertools

# models = {'resnet_fpn':resnet_fpn, 'densenet_fpn':densenet_fpn, 'mobilenet_fpn':mobilenet_fpn,
#           'resnet_unet':resnet_unet, 'densenet_unet':densenet_unet, 'mobilenet_unet':mobilenet_unet}
# models2 = list(itertools.product(models, repeat=2))
models2 = list(itertools.product(arch_dict, repeat=2))
metric = metrics[3]

for metric in metrics:
    #change to pandas
    p_vals = []
    for comb in models2:
        #print(comb[0], comb[1])
        t_stat, p = mannwhitneyu(arch_dict[comb[0]][metric], arch_dict[comb[1]][metric])
        p_vals.append(p)
    mat_format = int(len(p_vals)/len(arch_dict))
    p_vals = np.array(p_vals).reshape((mat_format,mat_format))

    import seaborn as sns

    plt.rcParams["figure.figsize"] = (12,12)

    mask = np.triu(np.ones_like(p_vals, dtype=bool))
    axis_labels = ['Resnet50+FPN', 'Densenet201+FPN', 'Mobilenetv2+FPN',
                            'Resnet50 + Unet', 'Densenet201 + Unet', 'Mobilenetv2+Unet']
    teste = (p_vals > 0.05)*1
    sns.set_theme(font_scale=1.2, style="white")
    sns.heatmap(teste, cmap=sns.cubehelix_palette(as_cmap=True), mask=mask,annot=False, 
                xticklabels=list(arch_dict.keys()), yticklabels=list(arch_dict.keys()), 
                fmt='.1e', cbar=False)
    plt.title(metric, fontsize =20)
    plt.show()

## Nemenyi test

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autorank import autorank, plot_stats, create_report, latex_table

In [ ]:

for arch in arch_dict:
    shapiro_test = stats.shapiro(arch_dict[arch]['F1-Score'])
    print(shapiro_test)
    plt.plot(arch_dict[arch]['F1-Score'])
    plt.show()

In [ ]:
for metric in metrics:
    df = pd.DataFrame()
    for arch in arch_dict:
        df[arch] = arch_dict[arch][metric]
    
    result = autorank(df, alpha=0.05, verbose=True, approach='frequentist')
    create_report(result)
    plot_stats(result)
    plt.title(metric, fontsize =20)
    plt.show()

In [ ]:
import itertools


#F1
# mr_dict = {
# 'MobilenetV2_MAnet_MR':15.300,
# 'Resnet50_MAnet_MR':13.100,
# 'Resnext101_FPN_MR':12.700,
# 'MobilenetV2_Unet_MR':12.100,
# 'MobilenetV2_FPN_MR':11.800,
# 'Resnext101_Unet_MR':10.500,
# 'Resnext101_MAnet_MR':10.000,
# 'Densenet201_FPN_MR':9.200,
# 'Resnext101_Unet++_MR':8.100,
# 'Densenet201_MAnet_MR':7.500,
# 'Resnet50_FPN_MR':7.000,
# 'Resnet50_Unet++_MR':4.900,
# 'MobilenetV2_Unet++_MR':4.400,
# 'Resnet50_Unet_MR':4.100,
# 'Densenet201_Unet_MR':3.000,
# 'Densenet201_Unet++_MR':2.300
#    }

mr_dict = {
'Resnext101_Unet++_MR':14.800,
'MobilenetV2_FPN_MR':14.600,
'Resnext101_MAnet_MR':14.400,
'Densenet201_FPN_MR':14.200,
'Densenet201_Unet++_MR':11.700,
'Resnet50_Unet++_MR':10.700,
'Densenet201_MAnet_MR':9.900,
'MobilenetV2 MAnet_MR':8.700,
'Densenet201_Unet_MR':7.100,
'MobilenetV2_Unet++_MR':6.200,
'Resnet50_FPN_MR':5.700,
'Resnet50_UnetMR':4.600,
'MobilenetV2_Unet_MR':4.300,
'Resnet50_MAnet_MR':4.100,
'Resnext101_FPN_MR':2.600,
'Resnext101_Unet_MR':2.400
    }
    
CD = 7.295

prev = None
i = 0
aux2 = list(itertools.permutations(mr_dict, r=2))

for comb in aux2:
    if abs(mr_dict[comb[1]]-mr_dict[comb[0]]) > CD:
        print(comb, abs(mr_dict[comb[1]]-mr_dict[comb[0]]))
    
    if comb[0] == prev and abs(mr_dict[comb[1]]-mr_dict[comb[0]]) > CD:
        i += 1
    elif comb[0] != prev:
        print(i)
        prev = comb[0]
        i = 0

In [ ]:
for metric in metrics:
    df = pd.DataFrame()
    for arch in arch_dict:
        df[arch] = arch_dict[arch][metric]
    
    result = autorank(df, alpha=0.05, verbose=True, approach='frequentist')
    plot_stats(result)
    plt.title(metric, fontsize =20)
    plt.show()

## Training x Testing time

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

color_list = ['#E0BBE4','#957DAD', '#FFE096', '#BED8D7', '#FF9356', '#7E74D4',
              '#9EE362', '#C33C23', '#FF6961', '#FFB447', '#95C7F1', '#E68AA9',
              '#658C72', '#2C1608', '#9B01F2', '#A6CE39']

for arch,color in zip(arch_dict,color_list):
    plt.scatter(arch_dict[arch]['Training Time'], arch_dict[arch]['Testing Time'], color=color)
plt.xlabel('Training Time (s)')
plt.ylabel('Testing Time (s)')
plt.legend(list(arch_dict.keys()), loc ="lower right")
plt.title('Training x Testing Time', fontsize=20)
plt.show()

In [ ]:
for arch in arch_dict:
    print(f"{arch} & {round(np.mean(arch_dict[arch]['Accuracy'])*100,2)} \u00B1 {round(np.std(arch_dict[arch]['Accuracy'])*100,2)} & {round(np.mean(arch_dict[arch]['F1-Score'])*100,2)} \u00B1 {round(np.std(arch_dict[arch]['F1-Score'])*100,2)} & {round(np.mean(arch_dict[arch]['Hausdorff']),2)} \u00B1 {round(np.std(arch_dict[arch]['Hausdorff']),2)} \\\\")         